In [ ]:
import pulsar

seed = b'43'
model = pulsar.REPOS[1] # celebahq

alice_stego = pulsar.Pulsar(seed=seed, repo=model)

In [ ]:
# Calculate the maximum message length that can be transmitted
m_len = alice_stego.estimate_regions(n_hist_bins=100, n_to_gen=1, end_to_end=True)
print(m_len)


In [8]:
# PICKLING CODE PROCESS **CHANGED*** 
import pulsar
import pickle

test = pulsar.REPOS[1] # celebahq
# Pickle the object to a binary file
with open("pickledFile.pickle", "wb") as file:
	pickle.dump(test, file)
    
print("Pickling completed")

#Include a Sanity Check


Pickling completed


In [9]:
# Generate a message to send
alice_message = b'E' * (m_len)

# Uncomment this for a random message instead
# import os
# alice_message = os.urandom(m_len)

alice_generate_results = alice_stego.generate_with_regions(alice_message)

NameError: name 'm_len' is not defined

In [ ]:
last = alice_stego.scheduler.num_inference_steps - 1
alice_hidden_sample = alice_generate_results["samples"][last]["hidden"]

In [ ]:
import PIL
import png # pypng package
import torch
import numpy as np

# TODO: Integrate this with Pulsar and its benchmarks

def display_sample(sample):
    image_processed = sample.cpu().permute(0, 2, 3, 1)
    image_processed = (image_processed + 1.0) * 127.5
    image_processed = image_processed.numpy().astype(np.uint8)

    image_pil = PIL.Image.fromarray(image_processed[0])

    display(image_pil)


def save_sample(sample, fname, dtype=np.uint16):
    image_processed = sample.cpu().permute(0, 2, 3, 1).numpy()
    dtype_max = np.iinfo(dtype).max

    y = image_processed[0]
    #   z = (dtype_max*((y - y.min())/y.ptp())).astype(dtype)
    z = (y + 1.0) * (dtype_max/2)
    image_processed = z.astype(dtype)

    # https://stackoverflow.com/a/25814423
    with open(fname, "wb") as f:
        writer = png.Writer(width=image_processed.shape[1], height=image_processed.shape[0], bitdepth=int(np.log2(dtype_max))+1, greyscale=False)
        image2list = image_processed.reshape(-1, image_processed.shape[1]*image_processed.shape[2]).tolist()
        writer.write(f, image2list)

def load_sample(fname, device, dtype=np.uint16):
    with open(fname, "rb") as f:
        reader = png.Reader(f)
        png_data  = reader.asDirect()
        # https://pypng.readthedocs.io/en/latest/ex.html#png-to-numpy-array-reading
        image = np.vstack(list(map(dtype, png_data[2])))

    dtype_max = np.iinfo(dtype).max

    image = image.reshape((1, 256, 256, 3))
    image = image.astype(np.float32)
    image = torch.tensor(image)
    image = (image / (dtype_max/2)) - 1.0

    sample = image.permute(0, 3, 1, 2)
    sample = sample.to(device)
    return sample

In [ ]:
fname = "/tmp/pulsar_alice.png"
alice_stego.save_sample(alice_hidden_sample, fname)
alice_hidden_sample

In [ ]:
display_sample(alice_hidden_sample)

In [ ]:
bob_stego = pulsar.Pulsar(seed=seed, repo=model)

In [ ]:
# Load sample from saved sample
# Uncomment if using a different filename
# fname = "/tmp/pulsar_bob.png"
received_hidden_sample = bob_stego.load_sample(fname)
received_hidden_sample

In [ ]:
display_sample(received_hidden_sample)

In [ ]:
# Sanity check
# Make sure Alice can decode before sending to Bob
alice_stego.reveal_with_regions(received_hidden_sample)


In [ ]:
# Offline phase
bob_stego.estimate_regions(n_hist_bins=100, n_to_gen=1, end_to_end=True)

In [ ]:
# Online phase
bob_stego.reveal_with_regions(received_hidden_sample)

In [ ]:
alice_stego.benchmarks

In [ ]:
bob_stego.benchmarks